In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
sentiment = pd.read_csv(r'D:\Desktop\sentiment analysis\data\raw\fear_greed_index.csv')
sentiment.head()

,timestamp,value,classification,date
0,1517463000,30,Fear,2018-02-01
1,1517549400,15,Extreme Fear,2018-02-02
2,1517635800,40,Fear,2018-02-03
3,1517722200,24,Extreme Fear,2018-02-04
4,1517808600,11,Extreme Fear,2018-02-05


In [3]:
trades = pd.read_csv(r'D:\Desktop\sentiment analysis\data\raw\historical_data.csv')
trades.head()

,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1.730000e+12
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,1.730000e+12
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050043,1.080000e+15,1.730000e+12
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.003055,1.050000e+15,1.730000e+12


In [4]:
#droppinng dupicates from both the datasets
trades.drop_duplicates(inplace=True)
sentiment.drop_duplicates(inplace=True)

In [5]:
print(trades.shape)
print(sentiment.shape)
print(trades.columns.to_list())
print(sentiment.columns.to_list())

(211224, 16)
(2644, 4)
['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side', 'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL', 'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID', 'Timestamp']
['timestamp', 'value', 'classification', 'date']


In [6]:
# normalize time columns
# trades: assume column 'time' or 'timestamp' exists (adjust name)
# 1. Convert and clean trade timestamps
# Note: Using .copy() prevents SettingWithCopyWarning if 'trades' is a slice
trades = trades.copy()
trades['Timestamp IST'] = pd.to_datetime(trades['Timestamp IST'], utc=True, errors='coerce')

# Drop rows where conversion failed
trades = trades.dropna(subset=['Timestamp IST'])

# Extract date for merging
trades['date'] = trades['Timestamp IST'].dt.date

# 2. Process sentiment data
sentiment_copy = sentiment.copy()
sentiment_copy['date'] = pd.to_datetime(sentiment['date'], errors='coerce').dt.date


# Drop duplicates to ensure a 1:1 or N:1 merge (Trade:Sentiment)
# We keep the first sentiment record per day
sent_clean = sentiment_copy.dropna(subset=['date']).drop_duplicates(subset=['date'])[['date', 'value', 'classification']]

In [7]:
sent_clean.head()

,date,value,classification
0,2018-02-01,30,Fear
1,2018-02-02,15,Extreme Fear
2,2018-02-03,40,Fear
3,2018-02-04,24,Extreme Fear
4,2018-02-05,11,Extreme Fear


In [8]:
print(sent_clean.isna().sum())
print(trades.isna().sum())


date              0
value             0
classification    0
dtype: int64
Account             0
Coin                0
Execution Price     0
Size Tokens         0
Size USD            0
Side                0
Timestamp IST       0
Start Position      0
Direction           0
Closed PnL          0
Transaction Hash    0
Order ID            0
Crossed             0
Fee                 0
Trade ID            0
Timestamp           0
date                0
dtype: int64


In [9]:
# define profit flag
trades['profit'] = trades['Closed PnL'] > 0

agg = trades.groupby(['Account', 'date']).apply(
    lambda g: pd.Series({
        'daily_pnl': g['Closed PnL'].sum(),
        'n_trades': len(g),
        'n_wins': (g['Closed PnL'] > 0).sum(),
        'win_rate': (g['Closed PnL'] > 0).mean(),
        'avg_trade_size_usd': g['Size USD'].abs().mean(),
        'total_volume_usd': g['Size USD'].abs().sum(),
        'longs': (g['Side'].str.lower() == 'buy').sum(),
        'shorts': (g['Side'].str.lower() == 'sell').sum()
    })
).reset_index()


In [10]:
agg.columns.tolist()

['Account',
 'date',
 'daily_pnl',
 'n_trades',
 'n_wins',
 'win_rate',
 'avg_trade_size_usd',
 'total_volume_usd',
 'longs',
 'shorts']

In [11]:
agg['date'] = pd.to_datetime(agg['date']).dt.date
sent_clean['date'] = pd.to_datetime(sent_clean['date']).dt.date

In [12]:
merged = pd.merge(
    agg,
    sent_clean,
    on='date',
    how='left'
)

merged.head()

,Account,date,daily_pnl,n_trades,n_wins,win_rate,avg_trade_size_usd,total_volume_usd,longs,shorts,value,classification
0,0x083384f897ee0f19899168e3b1bec365f52a9012,2024-06-12,-175611.000056,33.0,0.0,0.000000,36736.362424,1212299.96,33.0,0.0,72.0,Greed
1,0x083384f897ee0f19899168e3b1bec365f52a9012,2024-11-11,0.000000,177.0,0.0,0.000000,5089.718249,900880.13,0.0,177.0,76.0,Extreme Greed
2,0x083384f897ee0f19899168e3b1bec365f52a9012,2025-01-03,9482.221441,45.0,36.0,0.800000,2985.797556,134360.89,36.0,9.0,74.0,Greed
3,0x083384f897ee0f19899168e3b1bec365f52a9012,2025-02-02,76710.000000,10.0,9.0,0.900000,185847.000000,1858470.00,9.0,1.0,60.0,Greed
4,0x083384f897ee0f19899168e3b1bec365f52a9012,2025-03-02,101011.685664,839.0,256.0,0.305125,23593.019857,19794543.66,350.0,489.0,26.0,Fear


In [13]:
merged.columns.tolist()

['Account',
 'date',
 'daily_pnl',
 'n_trades',
 'n_wins',
 'win_rate',
 'avg_trade_size_usd',
 'total_volume_usd',
 'longs',
 'shorts',
 'value',
 'classification']

In [14]:
merged.isna().sum()

Account                 0
date                    0
daily_pnl               0
n_trades                0
n_wins                  0
win_rate                0
avg_trade_size_usd      0
total_volume_usd        0
longs                   0
shorts                  0
value                 377
classification        377
dtype: int64

In [15]:
print("Trader date range:")
print(merged['date'].min(), "to", merged['date'].max())

print("\nSentiment date range:")
print(sent_clean['date'].min(), "to", sent_clean['date'].max())

Trader date range:
2023-01-05 to 2025-12-04

Sentiment date range:
2018-02-01 to 2025-05-02


In [16]:
merged = pd.merge(
    agg,
    sent_clean,
    on='date',
    how='inner'
)

In [17]:
merged.isna().sum()

Account               0
date                  0
daily_pnl             0
n_trades              0
n_wins                0
win_rate              0
avg_trade_size_usd    0
total_volume_usd      0
longs                 0
shorts                0
value                 0
classification        0
dtype: int64

In [18]:
import os
os.getcwd()

'D:\\Desktop\\sentiment analysis\\main code'

In [19]:
merged.to_csv(
    "../data/processed/merged_trader_sentiment.csv",
    index=False
)
